<a href="https://colab.research.google.com/github/ovokojo/fintequity/blob/main/How%20to%20Analyze%20Stocks%20with%20AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README
This notebook aims to explore possibilities of AI in investment analysis. It uses various APIs to pull data and perform analyses. To ensure a smooth experience, please follow the setup instructions below.

**Pre-requisites**

Before running this notebook, you will need access to the following APIs:


*   **SEC-API** for accessing financial statements and other disclosures from the U.S. Securities and Exchange Commission. Get a free API key at: https://sec-api.io/

*   **OpenAI API** for utilizing advanced AI models for data analysis, summaries, and more. Get an API key at: https://platform.openai.com/



**Getting Started**

Configure API Keys: Store your SEC-API and OpenAI API keys in a secure location. This notebook uses Colab Secrets.

Run the notebook: Execute the cells in order, making sure to input your API keys or modify the code as neccessary to access them.



In [23]:
# !pip install -q sec-api
# !pip install openai

from google.colab import userdata
import openai
from openai import OpenAI
import requests

client = OpenAI(
    api_key=userdata.get("OAI_KEY"),
)
SEC_KEY = userdata.get("SEC_KEY")

In [18]:
# Query API - Search and filter all 18+ million filings and exhibits published on SEC EDGAR since 1993
def getLatest10K(ticker: str):
    """
    Retrieves the most recent Form 10-K filing for a given stock ticker symbol.

    Args:
    - ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.

    Returns:
    - dict: A dictionary containing the JSON response with the latest Form 10-K filing
            information if the request is successful (HTTP status code 200).
    - int: The HTTP status code of the response if the request is unsuccessful.
    """

    endpoint = f"https://api.sec-api.io?token={API_KEY}"
    params = {
        "query": {
          "query_string": {
            "query":  f"ticker:{ticker} AND formType:\"10-K\""
          }
        },
        "from": "0",
        "size": "1",
        "sort": [{ "filedAt": { "order": "desc" }}]
    }
    response = requests.post(endpoint, json=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        return response.status_code

# Example
# latest10K = getLatest10K('TSLA')

In [32]:
# Extractor API - Extract any text section from 10-Q, 10-K, and 8-K SEC filings.
def getLatest10kSection(ticker: str, section: str):
    """
    Retrieves a specified section from the latest 10-K filing of any stock using the SEC Query API & the extractor API

    Args:
        ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.
        section (str): Specific section of the filing to retrieve (e.g., "1A" for the Risk Factors section).
        10-K supported section codes: 1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15

    Returns:
        text or int: The extracted content in JSON format if request is successful; otherwise, the HTTP status code.
    """

    latest10K = getLatest10K(ticker)
    url_10k = latest10K['filings'][0]['linkToFilingDetails']

    endpoint = "https://api.sec-api.io/extractor"

    params = {
        "token": API_KEY,
        "url": url_10k,
        "item": section,
        "type": "text"
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with code: {response.status_code}")
        return response.status_code

# Example
# getLatest10kSection("WDAY", "1A")

In [34]:
# Next steps: analyze & summarize section with AI
ticker = "WDAY"
section = "1A"
data = getLatest10kSection(ticker, section)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Summarize the the key risk factors in section {section} of {ticker} 10-K: {data}",
        }
    ],
    model="gpt-4-turbo-preview",
)

response = chat_completion.choices[0].message.content

print(response)

Due to the length and complexity of the document provided, I have summarized the key risk factors from section 1A of the WDAY 10-K report into several core thematic areas. These summarized risk factors are designed to provide an overview of the most critical challenges and uncertainties that the company faces, which could potentially impact its business operations, financial condition, and market price of its securities.

1. **Information Technology and Infrastructure Risks**: Risks related to compromised IT systems, unauthorized access to customer or user data, service outages, delays in application deployment, and the failure of technical operations infrastructure can materially affect the company's operations and reputation.

2. **Regulatory and Privacy Risks**: The company faces substantial risks from privacy concerns, evolving domestic or foreign laws, and regulations regarding data protection and cross-border data transfer. Compliance with these evolving regulations could incur s